<a href="https://colab.research.google.com/github/AnitaTasnim/WrongWayDrivingCode/blob/main/WrongWayDriving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.0 MB/s eta 0:00:00


In [ ]:
!pip install cvzone

  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26298 sha256=d6f4d954950f0408d605684198bd34083ede47d6412f9c78e650f4f2a0006bec
  Stored in directory: /root/.cache/pip/wheels/5d/21/e8/3147ae88d44e27f06e0175d337a7673c70fb957202cbbe2034
Successfully built cvzone


In [ ]:
import math

class Tracker:
    def __init__(self):
        self.center_points = {}  # id -> (cx, cy)
        self.id_count = 0

    def update(self, objects_rect):
        objects_bbs_ids = []

        for rect in objects_rect:
            x1, y1, x2, y2 = rect

            # Correct center calculation
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
                    objects_bbs_ids.append([x1, y1, x2, y2, id])
                    same_object_detected = True
                    break

            if not same_object_detected:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x1, y1, x2, y2, self.id_count])
                self.id_count += 1

        # Remove lost IDs
        new_center_points = {}
        for _, _, _, _, object_id in objects_bbs_ids:
            new_center_points[object_id] = self.center_points[object_id]

        self.center_points = new_center_points
        return objects_bbs_ids


In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
import cvzone
import numpy as np
#from tracker import Tracker   # make sure this is imported

model = YOLO('yolov8s.pt')

cap = cv2.VideoCapture('wrongway.mp4')

# Get original video properties
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

# Output video writer
out = cv2.VideoWriter(
    'output_wrongway.mp4',
    cv2.VideoWriter_fourcc(*'mp4v'),
    fps,
    (1020, 500)   # must match resize
)

# Load class names
with open("coco.txt", "r") as f:
    class_list = f.read().split("\n")

tracker = Tracker()

area1 = [(593,227),(602,279),(785,274),(774,220)]
area2 = [(747,92),(785,208),(823,202),(773,95)]

count = 0
wup = {}
wrongway = []
prev_center = {}
motion_sum = {}


while True:
    ret, frame = cap.read()
    if not ret:
        break

    count += 1
    if count % 2 != 0:
        continue

    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame, verbose=False)
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype("float")

    detections = []

    for _, row in px.iterrows():
        x1, y1, x2, y2 = map(int, row[:4])
        cls_id = int(row[5])
        cls_name = class_list[cls_id]

        if 'car' in cls_name:
            detections.append([x1, y1, x2, y2])

    bbox_idx = tracker.update(detections)


    for x1, y1, x2, y2, obj_id in bbox_idx:
      cx = (x1 + x2) // 2
      cy = (y1 + y2) // 2


      if obj_id not in prev_center:
          prev_center[obj_id] = (cx, cy)
          motion_sum[obj_id] = 0
      else:
          px, py = prev_center[obj_id]
          dy = cy - py
          prev_center[obj_id] = (cx, cy)
          motion_sum[obj_id] += dy



      in_a1 = cv2.pointPolygonTest(np.array(area1, np.int32), (cx, cy), False)
      in_a2 = cv2.pointPolygonTest(np.array(area2, np.int32), (cx, cy), False)

      # NORMAL RULE
      if in_a1 >= 0:
          wup[obj_id] = True

      if obj_id in wup and in_a2 >= 0:
          if obj_id not in wrongway:
              wrongway.append(obj_id)


      if in_a2 >= 0 and motion_sum.get(obj_id, 0) < -15:
          if obj_id not in wrongway:
              wrongway.append(obj_id)


      # DRAW
      if obj_id in wrongway:
          cv2.rectangle(frame, (x1,y1), (x2,y2), (0,0,255), 3)
          cvzone.putTextRect(
              frame, "WRONG WAY", (x1, y1-10),
              scale=1.5, thickness=2, colorR=(0,0,255)
          )
      else:
          cv2.rectangle(frame, (x1,y1), (x2,y2), (255,0,255), 2)
          cvzone.putTextRect(frame, f'ID {obj_id}', (x1, y1), 1, 1)






    w = len(wrongway)
    cv2.polylines(frame, [np.array(area1, np.int32)], True, (255,255,255), 2)
    cv2.polylines(frame, [np.array(area2, np.int32)], True, (255,255,255), 2)
    cvzone.putTextRect(frame, f'wrongwaycar {w}', (60, 100), 2, 2)
    # WRITE FRAME TO FILE (instead of imshow)
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
